# ワクワクメールのデータ分析

In [129]:
import pandas as pd
import datetime
import seaborn as sns
import japanize_matplotlib

In [130]:
TOKYO_FILE_PATH = './../rawdata/wakuwaku_tokyo.csv'
KANAGAWA_FILE_PATH = './../rawdata/wakuwaku_kanagawa.csv'

tokyo_df = pd.read_csv(TOKYO_FILE_PATH)
kanagawa_df = pd.read_csv(KANAGAWA_FILE_PATH)

In [131]:
(len(tokyo_df), len(kanagawa_df))

(836, 361)

In [132]:
df = pd.merge(tokyo_df, kanagawa_df, how="outer")
df = df.drop(["image_url", "url"], axis=1)

In [133]:
print(len(df))
df.head()

1197


,age,city,genre,id,name,post_at,prefecture,title
0,20代前半,豊島区,すぐ会いたい,136637564,ちゃいか,1/1 15:46,東京都,処女なんです。卒業したいです
1,20代半ば,新宿区,すぐ会いたい,136642018,ゆり,1/1 15:46,東京都,これから歌舞伎町
2,20代前半,新宿区,すぐ会いたい,136644387,あおたん,1/1 15:44,東京都,えっちしよ？
3,20代前半,千代田区,すぐ会いたい,136646251,ゆりりん♪꒰｡･ω･`;꒱*:･,1/1 15:40,東京都,ご飯やデート～コスプレやグラビア撮影、美術モデルなど
4,20代半ば,江戸川区,すぐ会いたい,136643391,ちーちゃん,1/1 15:39,東京都,今日の髪型は～……！


## データ分析

### 地域別

In [134]:
output = pd.DataFrame(df.groupby("city").count()["id"])
output.index.name = "地域"
output = output.rename(columns = {"id": "投稿数"})
output.sort_values("投稿数", ascending=False).head(10)

,投稿数
地域,
新宿区,258
豊島区,158
横浜市西区,50
台東区,35
足立区,34
渋谷区,33
横浜市中区,33
品川区,24
川崎市川崎区,22


### 年代別

In [135]:
output = pd.DataFrame(df.groupby("age").count()["id"])
output.index.name = "年代"
output = output.rename(columns = {"id": "投稿数"})
output.sort_values("投稿数", ascending=False).head(20)

,投稿数
年代,
20代前半,473
20代半ば,194
18-19歳,144
20代後半,131
30代前半,101
30代半ば,56
30代後半,38
40代前半,20
40代半ば,16


### 時間帯別

In [157]:
def to_datetime(x):
    try:
        return datetime.datetime.strptime(x, '%Y/%m/%d %H:%M')                                        
    except Exception:
        now = datetime.datetime.now()
        post_at = datetime.datetime.strptime(x, '%m/%d %H:%M')
        if now.month == 1 and post_at.month == 12:
            post_at = post_at.replace(year=now.year - 1)
        else:
            post_at = post_at.replace(year=now.year)
        return post_at

output = df.assign(datetime=df.apply(lambda x: to_datetime(x.post_at), axis=1))
output = pd.DataFrame(output.groupby(output.datetime.map(lambda t: t.hour)).count()["id"])
output.index.name = "時間帯"
output = output.rename(columns = {"id": "投稿数"})
output.head(24)

,投稿数
時間帯,
0,31
1,34
2,22
3,20
4,12
5,17
6,10
7,17
8,27


## 検索

### キーワード検索

In [117]:
keyword = "会いたい"
df[df['title'].str.contains(keyword)]

,age,city,genre,id,name,post_at,prefecture,title
51,30代半ば,墨田区,すぐ会いたい,136641336,あい,1/1 14:39,東京都,会いたいな
102,20代後半,渋谷区,すぐ会いたい,136645260,ゆい,1/1 12:00,東京都,来年3日か4日に渋谷で特別な関係で会いたいです。
202,30代前半,武蔵野市,すぐ会いたい,136621739,りさ,2019/12/31 16:17,東京都,吉祥寺で会いたいな(*´ω`*)
598,20代前半,豊島区,すぐ会いたい,136535962,あきこ,2019/12/28 16:32,東京都,今日。会いたい。本あり。礼あり。
902,20代前半,横浜市青葉区,すぐ会いたい,136627310,ミミ,2019/12/31 22:48,神奈川県,会いたい！
907,20代前半,川崎市中原区,すぐ会いたい,136633446,えな,2019/12/31 21:10,神奈川県,こんにちは～\r\n今から、武蔵小杉の近くに会いたい
912,20代後半,横浜市中区,すぐ会いたい,136611919,まいこ,2019/12/31 18:49,神奈川県,日の出町にいます。すぐ会いたいです
943,20代前半,川崎市高津区,すぐ会いたい,0b3ddd95,莉乃,2019/12/31 5:50,神奈川県,会いたい
975,20代前半,横浜市港北区,すぐ会いたい,136601581,あみ,2019/12/30 19:54,神奈川県,これから車内かるめで会いたい@綱島
976,20代前半,大和市,すぐ会いたい,136603528,れな,2019/12/30 19:33,神奈川県,今から会いたい！


### 地域検索

In [118]:
area = "川崎市中原区"
df[df['city'].str.contains(area)]

,age,city,genre,id,name,post_at,prefecture,title
837,40代半ば,川崎市中原区,すぐ会いたい,136649481,めぐみ,1/1 15:31,神奈川県,内容みてね。します？今日限定メール待ってます
898,40代後半,川崎市中原区,すぐ会いたい,136629585,ななみ,2019/12/31 23:51,神奈川県,今から徒歩じゃない人、エッチな事で会えますよ！
907,20代前半,川崎市中原区,すぐ会いたい,136633446,えな,2019/12/31 21:10,神奈川県,こんにちは～\r\n今から、武蔵小杉の近くに会いたい
1112,40代前半,川崎市中原区,すぐ会いたい,136492336,らむ,2019/12/27 16:34,神奈川県,メンズエステ
